The most important key building block in `LangChain` is the `chain`.
A `chain` usually combines an LLM with a prompt, but also allows us to combine other chains and perform several operations on text.
One of the powers of a `chain` is running it over many inputs at a time.

# `LLMChain`

The `LLMChain` is a simple, but powerful chain that underpins a lot of the chains covered in this lesson.

In [1]:
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
api_version = "2023-12-01-preview"
model = "gpt-35-turbo-16k"

We initialize our model with a high temperature to elicit more randomness ("creativity") in the response.

In [3]:
chat = AzureChatOpenAI(model=model, temperature=0.9, api_version=api_version)

In [4]:
prompt = ChatPromptTemplate.from_template(
    template="What is the best name to describe a company that makes {product}?"
)

We combine the prompt and the model using the `LLMChain`.

In [5]:
chain = LLMChain(prompt=prompt, llm=chat)

We can run the chain by providing an input to the `run` method.

In [6]:
product = "Queen Size Sheet Set"
chain.run(product)

'The Royal Rest Linens'

# `SimpleSequentialChain`

The `LLMChain` is the most simple, basic type of chain in `LangChain`.
The `SimpleSequentialChain` runs a sequence of chains one after another.
This works well when we have chains that expect one input and return one output.

In [7]:
from langchain.chains import SimpleSequentialChain

To demonstrate, we'll treat our prior `LLMChain` as our first chain in a sequence of chains.

In [8]:
chain_one = chain.copy()

Now we create a second chain to take the output of the first chain (`chain_one`), a company name, and pass it into a second chain.

In [9]:
second_prompt = ChatPromptTemplate.from_template(
    template="Write a 20 word description for the following company: {company_name}"
)

chain_two = LLMChain(prompt=second_prompt, llm=chat)

We supply both chains to `SimpleSequentialChain` in the order we want to run.

In [10]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

We run the `simple_chain` like any other chain using the `run` method.
Remember that we are inputting a product as that is what the first chain (`chain_one`) expects.

In [11]:
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


AttributeError: 'LLMChain' object has no attribute 'callbacks'

Nearly 200 versions of `LangChain` have come out since the the DeepLearningAI tutorial was created.
After some toying I found that I needed to supply `output_key="company_name"` to the `chain_one`.

In [12]:
chain_one = LLMChain(prompt=prompt, llm=chat, output_key="company_name")

This doesn't affect the output.

In [13]:
chain_one.run(product)

'RoyalRest Bedding'

In [14]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Rest Linens"
Royal Rest Linens offers luxurious and high-quality bedding products, ensuring a comfortable and peaceful sleep experience fit for royalty.

> Finished chain.


'Royal Rest Linens offers luxurious and high-quality bedding products, ensuring a comfortable and peaceful sleep experience fit for royalty.'